In [1]:
import sys, os

env_root = '/N/project/baby_vision_curriculum/pythonenvs/hfenv/lib/python3.10/site-packages/'
sys.path.insert(0, env_root)

# SCRIPT_DIR = os.path.realpath(os.path.dirname(inspect.getfile(inspect.currentframe()))) #os.getcwd() #
# print('cwd: ',SCRIPT_DIR)
#os.path.realpath(os.path.dirname(inspect.getfile(inspect.currentframe())))
# util_path = os.path.normpath(os.path.join(SCRIPT_DIR, '..', 'util'))
# sys.path.insert(0, util_path)
os.environ['OPENBLAS_NUM_THREADS'] = '38' #@@@@ to help with the num_workers issue
os.environ['OMP_NUM_THREADS'] = '1'  #10


import numpy as np
import torch, torchvision
# from torchvision import transforms as tr
from tqdm import tqdm
from pathlib import Path
# import math
import argparse
# import pandas as pd
# import warnings


import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp



# from PIL import Image
# from torch.utils.data import Dataset
import random
from copy import deepcopy

# device_index = 0  # Use 0 for the first GPU, 1 for the second, and so on
# Get the GPU properties

from ddputils import is_main_process, save_on_master, setup_for_distributed, AllReduce
from homeview import (_get_transform, get_fpathlist,
    get_fold, get_train_val_split, get_fpath2framelist, 
    TwoFrameDataset, get_group)
# from mask import MaskCollator as MBMaskCollator
# import vision_transformer as vit
# from tensors import trunc_normal_
# from helper import init_opt, load_checkpoint
# from mask import update_masks, apply_masks
from loggingtools import (
        CSVLogger,
        grad_logger,
        AverageMeter,
        gpu_timer)
import yaml
import torch.nn.functional as F

import logging
from functools import partial


In [2]:
def _adapt_model_simclr(model, n_features, n_out): 
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(n_features, n_out), 
        torch.nn.ReLU(), 
        torch.nn.Linear(n_out, n_out)) 
    _ = model.float()
    return model

def get_model(device, model_name='resnet50', pred_emb_dim=1024):
    xmodel = getattr(torchvision.models, model_name)()
#     xmodel = torchvision.models.resnet50(pretrained=False)
    n_features, n_out = 2*pred_emb_dim, pred_emb_dim
    xmodel = _adapt_model_simclr(xmodel, n_features, n_out)
    xmodel = xmodel.to(device)
    return xmodel

In [3]:
def get_untrained_model(
    model_name='resnet50', pred_emb_dim=1024):
    xmodel = getattr(torchvision.models, model_name)()
#     xmodel = torchvision.models.resnet50(pretrained=False)
    n_features, n_out = 1*pred_emb_dim, pred_emb_dim
    xmodel = _adapt_model_simclr(xmodel, n_features, n_out)
    # xmodel = xmodel.to(device)
    return xmodel

def init_model_from_checkpoint(model, checkpoint_path):
    # caution: model class
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

def get_model(args):
    
    encoder = get_untrained_model(model_name=args.architecture, pred_emb_dim=args.pred_emb_dim)
    
    load_path = args.init_checkpoint_path
    if load_path!='na':
        print('load_path:',load_path)
        encoder = init_model_from_checkpoint(encoder, load_path)
    encoder.fc = torch.nn.Identity() #@@@
    return encoder


In [5]:
class Args:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

In [6]:
device='cpu'
pred_emb_dim=768
args = Args(
    architecture='resnet18',
    pred_emb_dim=pred_emb_dim,
    init_checkpoint_path='na'
)
xmodel = get_model(args)

In [7]:
xmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
device='cpu'
pred_emb_dim=1024
model_name='resnet50'
xmodel = get_model(
        device=device,
        pred_emb_dim=pred_emb_dim,
        model_name=model_name)

In [19]:
xmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
for n, p in xmodel.named_parameters():
#     if (p.grad is not None):
    if n=='conv1.weight':
        print(1)
    elif n=='fc.0.weight':
        print(2)

#         print(2)

1
2


In [35]:
n = 10  # Replace 'n' with the desired size of the matrix

# Using nested list comprehensions to create the matrix
x = [
    [1 if i == j + 1 or i == j - 1 else 0 
     for j in range(n)] 
    for i in range(n)]

# Set the first and last elements of each row to 0
# for row in x:
#     row[0] = 0
#     row[-1] = 0

# Print the matrix
for row in x:
    print(row)


[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
